In [29]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
# Load the data
file_path = "fashion_2018_2022.xlsx"
data = pd.read_excel(file_path)

In [30]:
print(data.head())

   product_id    product_name  gender category     pattern  color age_group  \
0        1001    Biker Jacket    Male    Shirt   Geometric  White     25-35   
1        1002  Business Shirt    Male   Jacket  Polka Dots  Beige     18-24   
2        1003     Wool Jacket  Female    Dress   Geometric  Brown     18-24   
3        1004    Summer Dress    Male   Shorts       Plain  White     25-35   
4        1005    Casual Jeans    Male    Shirt       Plain  Beige     35-45   

   season   price   material  sales_count  reviews_count  average_rating  \
0  Spring   70.36  Synthetic           75             65             4.9   
1  Summer   91.59       Wool          296             25             3.5   
2     All  129.52       Wool           50             32             4.3   
3     All  116.01     Cotton          435             73             4.6   
4  Winter  125.48    Viscose           79             36             3.6   

   out_of_stock_times brand  discount last_stock_date  wish_list_cou

In [31]:
df = data[data['year_of_sale'].isin([2018, 2019, 2020, 2021])]

In [32]:
# Encode 'season' column
le_season = LabelEncoder()
df['season_code'] = le_season.fit_transform(df['season'])

# Select relevant columns and group by product, year, and month
monthly_sales = df.groupby(['product_name', 'year_of_sale', 'month_of_sale']).agg({
    'sales_count': 'sum',
    'average_rating': 'mean',
    'price': 'mean',
    'season_code': 'first'  # Take the first season_code, assuming it's constant within a month
}).reset_index()

# Display the first few rows of the processed dataset
print(monthly_sales.head())

  product_name  year_of_sale  month_of_sale  sales_count  average_rating  \
0  Beach Shirt          2018              1          424            3.50   
1  Beach Shirt          2018              3           78            3.30   
2  Beach Shirt          2018              4          789            4.20   
3  Beach Shirt          2018              5          445            3.55   
4  Beach Shirt          2018              6          304            4.70   

     price  season_code  
0   97.100            0  
1   95.850            0  
2  104.060            0  
3   78.315            4  
4   43.440            3  


C:\Users\RAMPALLI VAMSI\AppData\Local\Temp\ipykernel_26156\478149094.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['season_code'] = le_season.fit_transform(df['season'])


In [33]:
# Scale numerical features
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(monthly_sales[['sales_count', 'average_rating', 'price']])

# Create a DataFrame with scaled data
scaled_df = pd.DataFrame(scaled_data, columns=['sales_count', 'average_rating', 'price'])
scaled_df['product_name'] = monthly_sales['product_name']
scaled_df['year_of_sale'] = monthly_sales['year_of_sale']
scaled_df['month_of_sale'] = monthly_sales['month_of_sale']
scaled_df['season_code'] = monthly_sales['season_code']  # Include season_code

# Display the first few rows of the scaled dataset
print(scaled_df.head())

   sales_count  average_rating     price product_name  year_of_sale  \
0     0.278689           0.250  0.592149  Beach Shirt          2018   
1     0.020864           0.150  0.582490  Beach Shirt          2018   
2     0.550671           0.600  0.645932  Beach Shirt          2018   
3     0.294337           0.275  0.446990  Beach Shirt          2018   
4     0.189270           0.850  0.177498  Beach Shirt          2018   

   month_of_sale  season_code  
0              1            0  
1              3            0  
2              4            0  
3              5            4  
4              6            3  


In [25]:
def create_sequences(data, seq_length=12):
    xs = []
    ys = []
    for i in range(len(data) - seq_length):
        x = data[i:i + seq_length]
        y = data[i + seq_length, 0]  # Assuming 'sales_count' is the first column
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Split data by product
product_groups = scaled_df.groupby('product_name')

X_train_list = []
y_train_list = []

for name, group in product_groups:
    group = group.sort_values(['year_of_sale', 'month_of_sale'])
    group_data = group[['sales_count', 'average_rating', 'price', 'season_code']].values
    X, y = create_sequences(group_data)
    X_train_list.append(X)
    y_train_list.append(y)

X_train = np.concatenate(X_train_list)
y_train = np.concatenate(y_train_list)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (203, 12, 4)
y_train shape: (203,)


In [26]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define LSTM model
model = Sequential()
model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1))  # Output layer predicting sales_count

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=16)

# Save the trained model
model.save('sales_model.h5')

Epoch 1/100


C:\Users\RAMPALLI VAMSI\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.0766 
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0472
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0474
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0508
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0422
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0393
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0388
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0421
Epoch 9/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0385
Epoch 10/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0381
Epoch 11/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0374
Epoch 12/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0492
Epoch 13/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0428
Epoch 14/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0401
Epoch 15/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0408
Epoch 16/100
13

In [27]:
# Load the trained model (if not loaded)
# model = tf.keras.models.load_model('lstm_sales_model.h5')

# Get last 12 months data for prediction in 2023
last_year_data = scaled_df[scaled_df['year_of_sale'] == 2021]
last_year_data = last_year_data.groupby('product_name').tail(12)  # Get last 12 months for each product

future_predictions = []

for name, group in last_year_data.groupby('product_name'):
    group_data = group[['sales_count', 'average_rating', 'price', 'season_code']].values
    group_data = group_data.reshape(1, group_data.shape[0], group_data.shape[1])

    predictions = []
    for i in range(12):
        pred = model.predict(group_data)
        predictions.append(pred[0, 0])

        # Prepare input for the next prediction (update sales_count, keep season_code)
        new_entry = np.array([pred[0, 0], group_data[0, -1, 1], group_data[0, -1, 2], group_data[0, -1, 3]]).reshape(1, 1, 4)
        group_data = np.concatenate((group_data[:, 1:, :], new_entry), axis=1)

    future_predictions.append((name, predictions))

# Convert predictions back to the original scale
predicted_sales_counts = []
for name, preds in future_predictions:
    preds = np.array(preds).reshape(-1, 1)
    dummy_features = np.zeros((len(preds), 2))  # Assuming 2 dummy features for average_rating and price
    scaled_preds = np.concatenate((preds, dummy_features), axis=1)
    
    # Inverse transform with the scaler
    original_scale_preds = scaler.inverse_transform(scaled_preds)
    
    # Extract sales_count predictions
    original_scale_preds = original_scale_preds[:, 0]
    
    # Format and collect predictions
    predicted_sales_counts.extend([(name, month + 1, count) for month, count in enumerate(original_scale_preds)])

# Create DataFrame for predictions
output_df = pd.DataFrame(predicted_sales_counts, columns=['product_name', 'Month', 'Predicted_sales_count'])
output_df['Year'] = 2022

# Print and save predictions to CSV
print(output_df)
output_df.to_csv('sales_2022_monthly.csv', index=False)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━

In [39]:
test = data[data['year_of_sale'] == 2022]
required_columns = ['product_name', 'year_of_sale', 'month_of_sale', 'sales_count']
test = test[required_columns]

In [40]:
test

,product_name,year_of_sale,month_of_sale,sales_count
528,Beach Shirt,2022,1,182
529,Casual Jeans,2022,1,325
530,Casual Jeans,2022,1,252
531,Heels,2022,1,52
532,Casual Jeans,2022,1,295
...,...,...,...,...
655,Wool Jacket,2022,12,328
656,Summer Dress,2022,12,200
657,Blouse,2022,12,478
658,Biker Jacket,2022,12,125


In [49]:
sorted_data = test.sort_values(by='product_name')

In [50]:
sorted_data

,product_name,year_of_sale,month_of_sale,sales_count
528,Beach Shirt,2022,1,182
632,Beach Shirt,2022,10,313
547,Beach Shirt,2022,2,239
546,Beach Shirt,2022,2,69
612,Beach Shirt,2022,8,112
...,...,...,...,...
623,Wool Jacket,2022,9,259
627,Wool Jacket,2022,10,491
542,Wool Jacket,2022,2,190
637,Wool Jacket,2022,10,145


In [68]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# Load the actual and predicted datasets
predicted_data = pd.read_csv('sales_2022_monthly.csv')

# Ensure both datasets have the same column names for merging
test.rename(columns={'month_of_sale': 'month', 'product_name': 'product'}, inplace=True)
predicted_data.rename(columns={'Month': 'month', 'product_name': 'product','Predicted_sales_count':'sales_count'}, inplace=True)

# Merge the actual and predicted data on 'product' and 'month'
merged_data = pd.merge(test, predicted_data, on=['product', 'month'], suffixes=('_actual', '_predicted'))

mse = mean_squared_error(merged_data['sales_count_actual'], merged_data['sales_count_predicted'])

In [70]:
mse

34677.051688525935

In [72]:
mse = mean_absolute_error(merged_data['sales_count_actual'], merged_data['sales_count_predicted'])
mse

156.2972746121161

In [69]:
# Calculate mean of actual sales counts
mean_actual = merged_data['sales_count_actual'].mean()

# Calculate percentage accuracy
accuracy = 100 - (mse / (mean_actual * 100))
print(f'Accuracy: {accuracy:.2f}%')

Accuracy: 98.69%
